In [3]:
import yfinance as yf
from math import sqrt
import pandas as pd
import numpy as np 

In [4]:
# Columns
list_columns = ["Cumulative Returns", "Max Drawdown", "Sharpe Ratio", "Sortino Ratio", "Volatility"]

In [6]:
class Index:
    
    list_of_indices = []
    index_tickers = []

    
    def __init__(self, ticker, risk_free_rate):
        self.ticker = ticker
        self.risk_free_rate = risk_free_rate
        self.start_date = "2010-01-01"
        self.end_date = "2023-05-01"
        self.period = '1d'
        self.data = self.fetch_data()
        
        # OLHC DATA: 
        self.open = self.data['Open']
        self.close = self.data['Close']
        self.high = self.data['High']
        self.low = self.data['Low']
        
        #indices using instances: 
        Index.list_of_indices.append(self)
        
        # tickers using instances: 
        Index.index_tickers.append(self.ticker)
        
    def fetch_data(self):
        return yf.download(self.ticker, start=self.start_date, end=self.end_date, period=self.period, progress=False)
        
        


ftse100 = Index("^FTSE", 0.02)
nk225 = Index("^N225", 0.01)
sp500 = Index("^GSPC", 0.03)
djia = Index("^DJI", 0.03)
nifty = Index("^NSEI", 0.05)

In [ ]:
class Equity:
    
    list_of_equities = []
    equities_tickers = []
    
    def __init__(self, ticker, risk_free_rate):
        self.ticker = ticker
        self.risk_free_rate = risk_free_rate
        self.start_date = "2010-01-01"
        self.end_date = "2023-05-01"
        self.period = '1d'
        self.data = self.fetch_data()
        
        # OLHC DATA: 
        self.open = self.data['Open']
        self.close = self.data['Close']
        self.high = self.data['High']
        self.low = self.data['Low']
        
        # equities using instances: 
        Equity.list_of_equities.append(self)
        
        # tickers using instances: 
        Equity.equities_tickers.append(self.ticker)
        
    def fetch_data(self):
        return yf.download(self.ticker, start=self.start_date, end=self.end_date, period=self.period, progress=False)

apple = Equity("AAPL", 0.02)
amazon = Equity("AMZN", 0.03)
nestle = Equity("NSRGF", 0.01)
reliance = Equity("RELIANCE.NS", 0.05)
toyota = Equity("TM", 0.01)

In [ ]:
Equity.list_of_equities

In [ ]:
close_prices = ftse100.close
open_prices = ftse100.open

# Daily Returns and Cumulative Returns :- 
    returns = (Current Value - Initial Value) / Initial Value

Where:
- Current Value is the value of the investment at the end of the period.
- Initial Value is the value of the investment at the beginning of the period.


In [ ]:
def dailyReturns(asset):
    return (asset.close - asset.open)/ asset.open 


def cumulativeReturns(asset):
    
    # Calculating daily returns, for a trading day: 
    dailyReturn = dailyReturns(asset)
    
    # Calculate the cumulative returns by multiplication: 
    cumulativeReturn = ((1 + dailyReturn).prod() - 1)
    
    return cumulativeReturn

In [ ]:
dailyReturns(apple)

In [ ]:
cumulativeReturns(ftse100)

# Volatility :- 

      vol = σ√T 

where:

- v = volatility 
- σ =standard deviation 
- T = number of periods 

In [ ]:
def Volatility(asset, adjust = True):
    
    # number of trading days:
    T = asset.data.shape[0]
    
    # Standard Deviation 
    std = dailyReturns(asset).std()
    
    # volatility :
    if(adjust): 
        volatility = std*sqrt(T)
    else: 
        volatility = std
        
    return volatility

In [ ]:
Volatility(ftse100)

# Sharpe Ratio :- 

In [ ]:
# risk_free_rate = 0.01

def excessReturns(asset):
    return dailyReturns(asset) - asset.risk_free_rate/252

def sharpeRatio(asset):
    avg_xr = np.mean(excessReturns(asset))
    std_xr = np.std(excessReturns(asset))
    sharpe = avg_xr/std_xr
    return sharpe

In [ ]:
sharpeRatio(apple)

In [ ]:
dailyReturns(apple)

# Sortino Ratio :-

In [ ]:
def sortinoRatio(asset):
    
    avg_dr = dailyReturns(asset).mean()

    negative_daily_returns = dailyReturns(asset)[dailyReturns(asset) < 0]

    # downside_deviation 
    std_nr = negative_daily_returns.std()

    sortino = (avg_dr - asset.risk_free_rate/252) / std_nr
    
    return sortino

In [ ]:
sortinoRatio(apple)

# Max Drawdown :- 

In [ ]:
def maxDrawdown(asset):
    peak = asset.close[0]
    drawdown = 0.0 
    
    for price in asset.close:
        if price > peak:
            peak = price
        else: 
            curr_drawdown = (price - peak)/peak
            if curr_drawdown < drawdown:
                drawdown = curr_drawdown
    mdd = drawdown
    
    return mdd

In [ ]:
maxDrawdown(nk225)

## Dataframe :- 

In [ ]:
index_data_list = []
for index in Index.list_of_indices:
    
    index_data = {
            'Cumulative Returns': cumulativeReturns(index),
            'Max Drawdown': maxDrawdown(index),
            'Sharpe Ratio': sharpeRatio(index),
            'Sortino Ratio': sortinoRatio(index),
            'Volatility': Volatility(index)
        }
    index_data_list.append(index_data)

In [ ]:
df_index = pd.DataFrame(index_data_list, index=Index.index_tickers)
df_index

In [ ]:
equity_data_list = []
for equity in Equity.list_of_equities:
    
    equity_data = {
            'Cumulative Returns': cumulativeReturns(equity),
            'Max Drawdown': maxDrawdown(equity),
            'Sharpe Ratio': sharpeRatio(equity),
            'Sortino Ratio': sortinoRatio(equity),
            'Volatility': Volatility(equity)
        }
    equity_data_list.append(equity_data)
    
df_equity = pd.DataFrame(equity_data_list, index=Equity.equities_tickers)
df_equity

# Markdown :-

In [ ]:
from tabulate import tabulate

table1 = tabulate(df_index, headers='keys', tablefmt='pipe')
table2 = tabulate(df_equity, headers='keys', tablefmt='pipe')

combined_tables = table1 + '\n\n' + table2

with open('tables.md', 'w') as f:
    f.write(combined_tables)